# DCGAN + conditional DCGAN


Notebook created by [Daniel Fojo](https://www.linkedin.com/in/daniel-fojo/) for the [Postgraduate course in artificial intelligence with deep learning](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/) in [UPC School](https://www.talent.upc.edu/ing/) (2020). Updated by [Mariona Carós](https://es.linkedin.com/in/mariona-c-a7bb91105) in 2022.
Minor contributions by [Gerard I. Gállego](https://www.linkedin.com/in/gerard-gallego/) and [Juan José Nieto](https://www.linkedin.com/in/juan-jose-nieto-salas/) during 2022.

In this notebook you will learn about Generative Adversarial Networks by implementing a DCGAN to generate images from noise.

**Important:** Set the Colab environment to run on GPU

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets, utils
from PIL import Image
import numpy as np
import math
from IPython.display import display
from tqdm import tqdm
from itertools import cycle
from typing import Tuple

In [ ]:
seed = 123
np.random.seed(seed)
_ = torch.manual_seed(seed)
_ = torch.cuda.manual_seed(seed)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

**Defining the Hyper-parameters**

We now define the hyperparameters that are going to be used throughout the notebook to define the network.


In [ ]:
hparams = {
    'batch_size':128,
    'num_epochs':20,
    'learning_rate':0.0002,
    'betas':(0.5, 0.999),
    'noise_size':100,
    'num_val_samples':25,
    'num_classes':10,
    'num_input_channels':1,
}

### Dataset and Data Loader
Download and prepare dataset and datalaoder


In [ ]:
train_transforms = transforms.Compose([   
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=train_transforms,
)

dataloader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=hparams['batch_size'], 
    shuffle=True,
)

# DCGAN

First, lets define our simple generator network.

### Exercise 1: Generator

The generator takes random noise as input and gives an image as output. Your exercise is to create the generator model.

It should follow these guidelines:
* The input will be a vector with random noise of size `noise_size`
* You should first apply a fully connected with output size 512\*4\*4 (channels\*height\*width)
* Then you should apply 3 blocks of:
    * TransposedConvolution with kernel size 4, stride 2 and padding 1. For the first 2 blocks, the output channels should be 256 and 128. For the third block, the output channels should be the correct one to generate images of the dataset.
    * BatchNorm2d except for the last block.
    * ReLU activation for the first 2 blocks and Tanh for the third block.

**Hint**: Remember to use reshape where necessary

In [ ]:
class Generator(torch.nn.Module):
    
    def __init__(self, noise_size: int, num_input_channels: int):
        super().__init__()
      
        # TODO: Create the Fully connected layer using nn.Linear
        self.fc = ...

        # TODO: Create the first block
        self.convt1 = nn.Sequential(
            nn.ConvTranspose2d(..., bias=False), # (B, 256, 8, 8)
            nn.BatchNorm2d(...),
            nn.ReLU(),
        )

        # TODO: Create the second block
        self.convt2 = nn.Sequential(
            nn.ConvTranspose2d(..., bias=False), # (B, 128, 16, 16)
            nn.BatchNorm2d(...),
            nn.ReLU(),
        )

        # TODO: Create the third block using nn.Sequential with ConvTranspose2d, and activation
        self.convt3 = nn.Sequential(
            nn.ConvTranspose2d(..., bias=False), # (B, num_input_channels, 32, 32)
            nn.Tanh(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # TODO: Define the forward of the generator, x are random noise vectors (B, noise_size)
        ...
        x = x.reshape(...) # (B, channels, height, width)
        ...
        
        return x

Similarly, let's define a simple discriminator

### Exercise 2: Discriminator

The discriminator takes an image as input and classifies it between Real or Fake (1 or 0). Your exercise is to create the discriminator model.

It should follow these guidelines:
* The input will be an image of size `[num_input_channels, 32, 32]`
* You should apply 3 blocks of:
    * Convolution with kernel size 4, stride 2 and padding 1. The output channels should be 128, 256 and 512.
    * BatchNorm2d except for the first block.
    * LeakyReLU activation (alpha=0.2)
* Then you should apply a fully connected with input size 512\*4\*4 (channels\*height\*width) and the correct output size and activation for binary classification


**Hint**: Remember to use reshape/flatten where necessary

In [ ]:
class Discriminator(torch.nn.Module):
    
    def __init__(self, num_input_channels: int):
        super().__init__()
        
        # TODO: Create the first block
        self.conv1 = nn.Sequential(
            nn.Conv2d(..., bias=False), # (B, 128, 16, 16)
            nn.LeakyReLU(0.2),
        )

        # TODO: Create the second block
        self.conv2 = nn.Sequential(
            nn.Conv2d(..., bias=False), # (B, 256, 8, 8)
            nn.BatchNorm2d(...),
            nn.LeakyReLU(0.2),
        )

        # TODO: Create the third block
        self.conv3 = nn.Sequential(
            nn.Conv2d(..., bias=False), # (B, 512, 4, 4)
            nn.BatchNorm2d(...),
            nn.LeakyReLU(0.2),
        )

        # TODO: Create the fully connected block using nn.Sequential with Linear and activation
        self.fc = nn.Sequential(
            nn.Linear(...)
            nn.Sigmoid(), # Binary classification (real vs false)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # TODO: Define the forward of the discriminator, x are images (B, num_input_channels, 32, 32)
        ...
        x = x.reshape(...) # (B, channels * height * width)
        ...

        return x

In [ ]:
generator = Generator(hparams['noise_size'], hparams['num_input_channels']).to(device)
optimizer_g = torch.optim.Adam(generator.parameters(), lr=hparams['learning_rate'], betas=hparams['betas'])

discriminator = Discriminator(hparams['num_input_channels']).to(device)
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=hparams['learning_rate'], betas=hparams['betas'])

criterion = nn.BCELoss()

def init_weights(m):
    if type(m) in {nn.Conv2d, nn.ConvTranspose2d, nn.Linear}:
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if m.bias != None:
            torch.nn.init.constant_(m.bias, 0.0)
    if type(m) == nn.BatchNorm2d:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)

generator.apply(init_weights)
discriminator.apply(init_weights);


## Train function

### Exercise 3: Train

Complete the code. Take into account which labels should be used at each step of the training.

In [ ]:
def train_batch(
        real_samples: torch.Tensor, 
        generator: torch.nn.Module,
        discriminator: torch.nn.Module,
        optimizer_g: torch.optim,
        optimizer_d: torch.optim,
        ) -> Tuple[float, float]:

    generator.train()
    discriminator.train()
    
    bsz = real_samples.shape[0]
    
    # TODO: Define the labels for the real (ones) and fake (zeros) images of size (bsz, 1)
    label_real = ...
    label_fake = ...

    label_real = label_real.to(device)
    label_fake = label_fake.to(device)

    ####################
    # OPTIMIZE GENERATOR
    ####################

    # Reset gradients
    optimizer_g.zero_grad()

    # Generate fake samples
    z = torch.randn(bsz, hparams['noise_size'], device=device) 
    fake_samples = generator(z) 
    
    # Evaluate the generated samples with the discriminator
    predictions_g_fake = discriminator(fake_samples) 
    # Calculate error with respect to what the generator wants
    loss_g = criterion(predictions_g_fake, label_real) 
        
    # Backpropagate
    loss_g.backward() 
    
    # Update weights (do a step in the optimizer)
    optimizer_g.step() 
    
    ########################
    # OPTIMIZE DISCRIMINATOR
    ########################

    fake_samples = fake_samples.detach() # Let's detach them to freeze the generator

    # Reset gradients
    optimizer_d.zero_grad()

    # Calculate discriminator prediction for real samples
    predictions_d_real = discriminator(real_samples)

    # Calculate error with respect to what the discriminator wants
    loss_d_real = criterion(predictions_d_real, label_real) 

    # Calculate discriminator loss for fake samples
    predictions_d_fake = discriminator(fake_samples) 

    # Calculate error with respect to what the discriminator wants
    loss_d_fake = criterion(predictions_d_fake, label_fake) 
    
    # Total discriminator loss
    loss_d = (loss_d_real + loss_d_fake) / 2
    loss_d.backward()

    # Update weights (do a step in the optimizer)
    optimizer_d.step() 

    return loss_g.item(), loss_d.item()


## Evaluation function

In [ ]:
@torch.no_grad()
def evaluate(generator: torch.nn.Module, z_val: torch.Tensor) -> Image.Image:
    
    generator.eval()
    fake_samples = generator(z_val).cpu()
    # select a sample or create grid if img is a batch
    nrows = int(math.sqrt(fake_samples.shape[0]))
    img = utils.make_grid(fake_samples, nrow=nrows)

    # unnormalize
    img = (img*0.5 + 0.5)*255

    # to numpy
    image_numpy = img.numpy().astype(np.uint8)
    image_numpy = np.transpose(image_numpy, (1, 2, 0))
    
    return Image.fromarray(image_numpy)


## Train loop

In [ ]:
z_val = torch.randn(hparams['num_val_samples'], hparams['noise_size'], device=device)

for epoch in range(hparams['num_epochs']):

    for i, (real_samples, labels) in enumerate(dataloader):
        real_samples = real_samples.to(device)
        loss_g, loss_d = train_batch(real_samples, generator, discriminator, optimizer_g, optimizer_d)

        if (i+1) % 200 == 0:
            print(f"\nEpoch: {epoch+1}/{hparams['num_epochs']}, batch: {i+1}/{len(dataloader)},"
                  +f" G_loss: {loss_g}, D_loss: {loss_d}")
            
            fake_images = evaluate(generator, z_val)
            display(fake_images)

    print(f"\nEpoch: {epoch+1}/{hparams['num_epochs']}, batch: {i+1}/{len(dataloader)},"
          +f" G_loss: {loss_g}, D_loss: {loss_d}")
    
    fake_images = evaluate(generator, z_val)
    display(fake_images)
    

# Extra: Conditional GAN

### Exercise 4: Generator

We will now modify the generator from before to a conditional generator. To do it, we will concatenate the input to the convolutions with an embedding of the label we want to generate.

Complete the forward method. To do it, use the embedding layer with the label, and then use `torch.cat` to concatenate the label as a channel (after the corresponding `reshape`)

**Hint**: The embedding is concatenated as a new channel.

In [ ]:
class ConditionalGenerator(torch.nn.Module):
    
    def __init__(
            self, 
            noise_size: int,
            num_input_channels: int,
            num_classes: int
            ) -> None:
        
        super().__init__()
      
        # TODO: Create the Fully connected layer using nn.Linear
        self.fc = ...

        # Embedding of the class
        self.embedding = nn.Embedding(
            num_embeddings=num_classes, 
            embedding_dim=4*4,
        )

        # TODO: Create the first block
        self.convt1 = nn.Sequential(
            nn.ConvTranspose2d(...+1, ..., bias=False), # (B, 256, 8, 8)
            nn.BatchNorm2d(...),
            nn.ReLU(),
        )

        # TODO: Create the second block
        self.convt2 = nn.Sequential(
            nn.ConvTranspose2d(..., bias=False), # (B, 128, 16, 16)
            nn.BatchNorm2d(...),
            nn.ReLU(),
        )

        # TODO: Create the third block using nn.Sequential with ConvTranspose2d, and activation
        self.convt3 = nn.Sequential(
            nn.ConvTranspose2d(..., bias=False), # (B, num_input_channels, 32, 32)
            nn.Tanh(),
        )

    def forward(self, x: torch.Tensor, label: int) -> torch.Tensor:
        
        # TODO: Define the forward of the generator, x are random noise vectors (B, noise_size)
        ...
        x = x.reshape(...) # (B, channels, height, width)
        emb = self.embedding(...).reshape(-1, 1, 4, 4) # (B, 1, height, width)
        x = torch.cat(..., dim=1)
        ...
        
        return x

### Exercise 5: Discriminator

We will now modify the discriminator from before to a conditional discriminator. To do it, we will concatenate the input image with an embedding of the label we want to generate.

Complete the forward method. To do it, use the embedding layer with the label, and then use `torch.cat` to concatenate the label as a channel (after the corresponding `reshape`)

**Hint**: The embedding is concatenated as a new channel.

In [ ]:
class ConditionalDiscriminator(torch.nn.Module):
    
    def __init__(self, 
                 num_input_channels: int,
                 num_classes: int):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=num_classes, 
            embedding_dim=32*32,
        )
        
        # TODO: Create the first block
        self.conv1 = nn.Sequential(
            nn.Conv2d(...+1, ..., bias=False), # (B, 128, 16, 16)
            nn.LeakyReLU(0.2),
        )

        # TODO: Create the second block
        self.conv2 = nn.Sequential(
            nn.Conv2d(..., bias=False), # (B, 256, 8, 8)
            nn.BatchNorm2d(...),
            nn.LeakyReLU(0.2),
        )

        # TODO: Create the third block
        self.conv3 = nn.Sequential(
            nn.Conv2d(..., bias=False), # (B, 512, 4, 4)
            nn.BatchNorm2d(...),
            nn.LeakyReLU(0.2),
        )

        # TODO: Create the fully connected block using nn.Sequential with Linear and activation
        self.fc = nn.Sequential(
            nn.Linear(...)
            nn.Sigmoid(), # Binary classification (real vs false)
        )

    def forward(self, x: torch.Tensor, label: int) -> torch.Tensor:
        # TODO: Define the forward of the discriminator, x are images (B, num_input_channels, 32, 32)
        emb = self.embedding(...).reshape(-1, 1, 32, 32) # (B, 1, 32, 32)
        x = torch.cat(..., dim=1)
        ...
        x = x.reshape(...) # (B, channels * height * width)
        ...
    

In [ ]:
generator = ConditionalGenerator(hparams['noise_size'], hparams['num_input_channels'], 
                                 hparams['num_classes']).to(device)
optimizer_g = torch.optim.Adam(generator.parameters(), lr=hparams['learning_rate'], 
                               betas=hparams['betas'])

discriminator = ConditionalDiscriminator(hparams['num_input_channels'],hparams['num_classes']).to(device)
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=hparams['learning_rate'], betas=hparams['betas'])

criterion = nn.BCELoss()

def init_weights(m):
    if type(m) in {nn.Conv2d, nn.ConvTranspose2d, nn.Linear}:
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if m.bias != None:
            torch.nn.init.constant_(m.bias, 0.0)
    if type(m) == nn.BatchNorm2d:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)

generator.apply(init_weights)
discriminator.apply(init_weights);


## Train function

In [ ]:
def train_batch_conditional(
        real_samples: torch.Tensor, 
        real_classes: torch.Tensor,
        generator: torch.nn.Module,
        discriminator: torch.nn.Module,
        optimizer_g: torch.optim,
        optimizer_d: torch.optim,
        ) -> Tuple[float, float]:

    generator.train()
    discriminator.train()

    bsz = real_samples.shape[0]

    # TODO: Define the labels for the real (ones) and fake (zeros) images of size (bsz, 1)
    label_real = ...
    label_fake = ...

    label_real = label_real.to(device)
    label_fake = label_fake.to(device)

    ####################
    # OPTIMIZE GENERATOR
    ####################

    # Reset gradients
    optimizer_g.zero_grad() 

    # Generate fake samples
    z = torch.randn(bsz, hparams['noise_size'], device=device) 
    fake_classes = torch.randint(0, 10, size=(bsz, 1), device=device)
    fake_samples = generator(z, fake_classes) 
    
    # Evaluate the generated samples with the discriminator
    predictions_g_fake = discriminator(fake_samples, fake_classes) 

    # Calculate error with respect to what the generator wants
    loss_g = criterion(predictions_g_fake, label_real) 

    # Backpropagate
    loss_g.backward() 
    
    # Update weights
    optimizer_g.step() 

    ########################
    # OPTIMIZE DISCRIMINATOR
    ########################

    fake_samples = fake_samples.detach()
    
    # Reset gradients
    optimizer_d.zero_grad() 

    # Calculate discriminator prediction for real samples
    predictions_d_real = discriminator(real_samples, real_classes) 

    # Calculate error with respect to what the discriminator wants
    loss_d_real = criterion(predictions_d_real, label_real) 

    # Calculate discriminator loss for fake samples
    predictions_d_fake = discriminator(fake_samples, fake_classes) 

    # Calculate error with respect to what the discriminator wants
    loss_d_fake = criterion(predictions_d_fake, label_fake) 
    
    # Total discriminator loss
    loss_d = (loss_d_real + loss_d_fake) / 2

    # Backpropagate
    loss_d.backward() 

    # Update weights
    optimizer_d.step() 

    return loss_g.item(), loss_d.item()


## Evaluation function

In [ ]:
@torch.no_grad()
def evaluate_conditional(
        generator: torch.nn.Module,
        z_val: torch.Tensor,
        labels_val: torch.Tensor,
        ) -> Image.Image:
    
    generator.eval()
    fake_samples = generator(z_val, labels_val).cpu()
    # select a sample or create grid if img is a batch
    nrows = int(math.sqrt(fake_samples.shape[0]))
    img = utils.make_grid(fake_samples, nrow=nrows)

    # unnormalize
    img = (img*0.5 + 0.5)*255

    # to numpy
    image_numpy = img.numpy().astype(np.uint8)
    image_numpy = np.transpose(image_numpy, (1, 2, 0))
    return Image.fromarray(image_numpy)


## Train loop

In [ ]:
z_val = torch.randn(hparams['num_val_samples'], hparams['noise_size'], device=device)
labels_cycle = cycle(range(hparams['num_classes']))
labels_val = torch.tensor([next(labels_cycle) for i in range(hparams['num_val_samples'])],
                          device=device).unsqueeze(1) #the labels will be a cycle from 0 to 9

for epoch in range(hparams['num_epochs']):

    for i, (real_samples, real_classes) in enumerate(dataloader):
        real_samples = real_samples.to(device)
        real_classes = real_classes.unsqueeze(1).to(device)
        loss_g, loss_d = train_batch_conditional(
            real_samples, 
            real_classes, 
            generator, 
            discriminator, 
            optimizer_g, 
            optimizer_d,
        )

        if (i+1) % 200 == 0:
            print(f"\nEpoch: {epoch+1}/{hparams['num_epochs']}, batch: {i+1}/{len(dataloader)},"
                  + f" G_loss: {loss_g}, D_loss: {loss_d}")
            
            fake_images = evaluate_conditional(generator, z_val, labels_val)
            display(fake_images)

    print(f"\nEpoch: {epoch+1}/{hparams['num_epochs']}, batch: {i+1}/{len(dataloader)}, "
          + f"G_loss: {loss_g}, D_loss: {loss_d}")
    
    fake_images = evaluate_conditional(generator, z_val, labels_val)
    display(fake_images)
    

### Conditional generation

In [ ]:
#You can play and visualize different numbers
number_chosen = 3
z_val = torch.randn(hparams['num_val_samples'], hparams['noise_size'], device=device)
number_chosen_torch = torch.tensor([
        number_chosen for i in range(hparams['num_val_samples'])
    ], 
    device=device).unsqueeze(1)

images_number_chosen = evaluate_conditional(generator, z_val, number_chosen_torch)
display(images_number_chosen)